## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [2]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="bitcoin"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('bitcoin_train_8_2.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,9.4545,10.3018,9.4297,10.1893,796676,USD
1,2020-06-02,10.1907,10.2073,9.3473,9.5276,1058556,USD
2,2020-06-03,9.5270,9.6672,9.4217,9.6672,558116,USD
3,2020-06-04,9.6672,9.8644,9.4909,9.7944,593664,USD
4,2020-06-05,9.7940,9.8461,9.6287,9.6312,527054,USD
...,...,...,...,...,...,...,...
580,2022-01-02,47.7387,47.9449,46.7182,47.3118,27020,USD
581,2022-01-03,47.2939,47.5560,45.7040,46.4302,41062,USD
582,2022-01-04,46.4357,47.5054,45.6021,45.8373,55589,USD
583,2022-01-05,45.8331,47.0194,42.5351,43.4259,83744,USD


Chuẩn hóa dữ liệu

In [3]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Open'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Open'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Open'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Open'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Open'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}_8_2_standardized.csv")
print(df)

           Date     Open     High      Low    Close  Volume Currency     H-L  \
20   2020-06-21   9.3588   9.4112   9.2885   9.2964  335994      USD  0.1227   
21   2020-06-22   9.2965   9.7519   9.2858   9.6837  523319      USD  0.4661   
22   2020-06-23   9.6839   9.7156   9.5818   9.6246  375494      USD  0.1338   
23   2020-06-24   9.6246   9.6674   9.2230   9.3020  515777      USD  0.4444   
24   2020-06-25   9.3008   9.3273   9.0226   9.2475  524453      USD  0.3047   
..          ...      ...      ...      ...      ...     ...      ...     ...   
580  2022-01-02  47.7387  47.9449  46.7182  47.3118   27020      USD  1.2267   
581  2022-01-03  47.2939  47.5560  45.7040  46.4302   41062      USD  1.8520   
582  2022-01-04  46.4357  47.5054  45.6021  45.8373   55589      USD  1.9033   
583  2022-01-05  45.8331  47.0194  42.5351  43.4259   83744      USD  4.4843   
584  2022-01-06  43.4316  43.7723  42.4811  43.0975   63076      USD  1.2912   

        O-C      SMA_7     SMA_14     S

Scale Miền giá trị lại

In [4]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Open']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))
# scaled_data_x
scaled_data_y

array([[0.00598767],
       [0.00492308],
       [0.011543  ],
       [0.01052968],
       [0.00499656],
       [0.00408576],
       [0.0025991 ],
       [0.        ],
       [0.00195488],
       [0.00303484],
       [0.00217873],
       [0.00378501],
       [0.00130895],
       [0.00100307],
       [0.00215139],
       [0.00123034],
       [0.0056459 ],
       [0.00423614],
       [0.00720605],
       [0.00394051],
       [0.00474024],
       [0.00385849],
       [0.00499143],
       [0.00401057],
       [0.00419   ],
       [0.00325186],
       [0.00217019],
       [0.00251878],
       [0.00276314],
       [0.00341249],
       [0.00262986],
       [0.00647639],
       [0.0086329 ],
       [0.01009564],
       [0.00919852],
       [0.01164724],
       [0.01579452],
       [0.03336791],
       [0.03246737],
       [0.03584055],
       [0.03568164],
       [0.03972639],
       [0.04774753],
       [0.03517583],
       [0.03785695],
       [0.03719051],
       [0.04659238],
       [0.046

In [5]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

Tách tập dữ liệu thành train và test

In [6]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}82.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 26s 56ms/step - loss: 0.0332
Epoch 2/60
40/40 [==============================] - 2s 58ms/step - loss: 0.0145
Epoch 3/60
40/40 [==============================] - 2s 59ms/step - loss: 0.0108
Epoch 4/60
40/40 [==============================] - 2s 58ms/step - loss: 0.0092
Epoch 5/60
40/40 [==============================] - 2s 59ms/step - loss: 0.0073
Epoch 6/60
40/40 [==============================] - 2s 59ms/step - loss: 0.0090
Epoch 7/60
40/40 [==============================] - 2s 59ms/step - loss: 0.0073
Epoch 8/60
40/40 [==============================] - 2s 62ms/step - loss: 0.0058
Epoch 9/60
40/40 [==============================] - 2s 61ms/step - loss: 0.0054
Epoch 10/60
40/40 [==============================] - 2s 60ms/step - loss: 0.0046
Epoch 11/60
40/40 [==============================] - 3s 64ms/step - loss: 0.0042
Epoch 12/60
40/40 [==============================] - 2s 62ms/step - loss: 0.0045
Epoch 13/60
40/40 [=================